In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
current_date = current_date.replace(year=2024)
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
    read_options={"timeout" : 600},

)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-05-10 16:54:56,923 INFO: Initializing external client
2025-05-10 16:54:56,930 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 16:54:59,012 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680
Fetching data from 2024-04-11 20:54:56.902426+00:00 to 2024-05-10 19:54:56.902426+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.48s) 


In [4]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features_and_target
features = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=24)

Skipping location_id 4757.06: Not enough data to create even one window.
Skipping location_id 4756.04: Not enough data to create even one window.


In [5]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-10 16:56:25,751 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 16:56:25,762 INFO: Initializing external client
2025-05-10 16:56:25,762 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 16:56:26,405 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680


Downloading: 0.000%|          | 0/330679 elapsed<00:00 remaining<?

In [6]:
features, target = features

In [7]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [8]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,3344.02,4.0,2024-05-10 21:00:00+00:00
1,3779.02,0.0,2024-05-10 21:00:00+00:00
2,3874.01,4.0,2024-05-10 21:00:00+00:00
3,3728.04,0.0,2024-05-10 21:00:00+00:00
4,3121.01,0.0,2024-05-10 21:00:00+00:00
...,...,...,...
563,3535.03,1.0,2024-05-10 21:00:00+00:00
564,4007.01,2.0,2024-05-10 21:00:00+00:00
565,4485.10,-0.0,2024-05-10 21:00:00+00:00
566,3019.02,0.0,2024-05-10 21:00:00+00:00


In [11]:
from src.inference import get_feature_store
feature_store = get_feature_store()
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-05-10 17:00:10,864 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 17:00:10,864 INFO: Initializing external client
2025-05-10 17:00:10,864 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 17:00:11,598 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680


In [12]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214680/fs/1203298/fg/1458537


Uploading Dataframe: 100.00% |██████████| Rows 568/568 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: citibike_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214680/jobs/named/citibike_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('citibike_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)